AutoPHoT Example script to highligh basic operations of package 

In [1]:
import autophot

AutoPhOT requires input based on input datalist of parameters
* Many of these are initial guesses and will be changed
* Need to load initial syntax

In [3]:
from autophot.prep_input import load
syntax = load()

Current build of AutoPhOT works of local computer
(Future builds will be accessed via website)
* can give directory of fits file via **fits_dir** or single file via **fname**

In [14]:
from autophot.example import load_example

ImportError: cannot import name 'load_example'